In [22]:
from langchain_community.document_loaders import PyPDFLoader, CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from typing_extensions import List, TypedDict
from typing import Iterator
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
import getpass
import os

In [2]:
cards_loader = CSVLoader("./data/cards.csv")
doc1_loader = PyPDFLoader("./data/doc1.pdf")
doc2_loader = PyPDFLoader("./data/doc2.pdf")

In [3]:
cards = cards_loader.load()
doc1 = doc1_loader.load()
doc2 = doc2_loader.load()

In [4]:
csv_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, chunk_overlap=20, add_start_index=True
)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50, add_start_index=True
)

In [5]:
card_splits = csv_splitter.split_documents(cards)
doc1_splits = text_splitter.split_documents(doc1)
doc2_splits = text_splitter.split_documents(doc2)

In [7]:
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

Enter API key for OpenAI:  ········


In [8]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = Chroma(embedding_function=embeddings, persist_directory="./vdb")

In [14]:
#_ = vector_store.add_documents(documents=card_splits)
print("Cards done")
#_ = vector_store.add_documents(documents=doc1_splits)
print("doc1 done")
#_ = vector_store.add_documents(documents=doc2_splits)
print("doc2 done")

Cards done
doc1 done
doc2 done


In [10]:
prompt = PromptTemplate.from_template(
    """
    Ты помощник-ассистент, который отвечает на вопросы клиента.
    Отвечай используя только следующий контекст.
    Если ты не знаешь или не нашел ответа, так и скажи.
    Если ты нашел несколько ответов, то расскажи о каждой из них.
    Вопрос: {question} 
    Контекст: {context} 
    Ответ:
    """
)

In [23]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [24]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    for doc in retrieved_docs:
        print(f"{doc.metadata}\n")
        print(f"{doc.page_content}\n")
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [28]:
result = graph.invoke({"question": "Когда родился Наполеон"})
print(result["answer"])

{'page': 38, 'page_label': '39', 'source': './data/doc1.pdf', 'start_index': 5042}

рабочих дней с даты наступления любого из указанных событий.

{'page': 54, 'page_label': '55', 'source': './data/doc1.pdf', 'start_index': 1425}

h. прохождения срочной военной службы, военных сборов.
5.2. Страховое покрытие по рискам «Смерть» и «Инвалидность» также не распространяется на события, произошедшие 
в результате проведения операции и (или) госпитализации, которые прямо связаны:
a. с врожденными аномалиями или возникшими вследствие них заболеваниями;
b. с переменой пола, со стерилизацией, лечением бесплодия, прерыванием беременности не по медицинским 
показаниям;

{'page': 53, 'page_label': '54', 'source': './data/doc1.pdf', 'start_index': 2576}

в отношении данного Застрахованного лица.
• Болезнь, следствием которой она явилась, диагностирована в Период страхования, установленный в отношении 
данного Застрахованного лица и не ранее Даты первичного подключения к Программе страхования (в случа